<a href="https://colab.research.google.com/github/vivek9048/data-science/blob/main/chatbot%20app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install langchain_groq langchain_core langchain_community

In [25]:
%%writefile my_chatbot_module.py
from langchain_groq import ChatGroq
llm=ChatGroq(
    temperature =0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
result=llm.invoke("who is vivek")
print(result.content)

Writing my_chatbot_module.py


In [4]:
!pip install pypdf

In [5]:
!pip install chromadb

In [6]:
!pip install sentence_transformers

In [7]:
!pip install langchain c

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.2/37.2 MB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = reso

In [29]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import gradio as gr

def initialize_llm():
  llm=ChatGroq(
    temperature =0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
  return llm
def create_vector_db():
  loader =DirectoryLoader("/content/lang/",glob= '*.pdf',loader_cls=PyPDFLoader)
  documents=loader.load()
  text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
  texts=text_splitter.split_documents(documents)
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma.from_documents(texts,embeddings,persist_directory = './chroma_db')
  vector_db.persist()
  print("ChromaDB created and data saved")

  return vector_db
  def setup_qa_chain(vector_db,llm):
    retirver=vector_db.as_retriever()
    prompt_templates= """ you are a companssionate mental health chatbot respond carefully:
    {context}
    user: {question}
    chatbot:"""
    prompt=PromptTemplate(template=prompt_templates,input_variables=['context','question'])

    qa_chain=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retirver,
        chain_type_kwargs={'prompt':prompt}
    )
    return qa_chain


def main():
  print("initializing chatbot.....")
  llm=initialize_llm()

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
  llm=ChatGroq(
    temperature =0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
  return llm
def create_vector_db():
  loader =DirectoryLoader("/content/lang/",glob= '*.pdf',loader_cls=PyPDFLoader)
  documents=loader.load()
  text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
  texts=text_splitter.split_documents(documents)
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma.from_documents(texts,embeddings,persist_directory = './chroma_db')
  vector_db.persist()
  print("ChromaDB created and data saved")

  return vector_db
def setup_qa_chain(vector_db,llm):
    retriever=vector_db.as_retriever()
    prompt_templates= """ you are a companssionate mental health chatbot respond carefully:
    {context}
    user: {question}
    chatbot:"""
    prompt=PromptTemplate(template=prompt_templates,input_variables=['context','question'])

    qa_chain=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        chain_type_kwargs={'prompt':prompt}
    )
    return qa_chain


def main():
  print("initializing chatbot.....")
  llm=initialize_llm()

  db_path="/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db=create_vector_db()
  else:
     embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
     vector_db=Chroma(persist_directory=db_path,embedding_function=embeddings)
  qa_chain=setup_qa_chain(vector_db,llm)

  while True:
    query=input("\nHuman:")
    if query.lower() == "exit":
      print("Chatbot: Take Care of yourself,Goodbye!")
      break
    response= qa_chain.run(query)
    print(f"Chatbot:{response}")

if __name__=="__main__":
  main()
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma(persist_directory=db_path,embedding_function=embeddings)
  qa_chain=setup_qa_chain(vector_db,llm)

  while True:
    query=input("\nHuman:")
    if query.lower() == "exit":
      print("Chatbot: Take Care of yourself,Goodbye!")
      break
    response= qa_chain.run(query)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
  llm=ChatGroq(
    temperature =0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
  return llm
def create_vector_db():
  loader =DirectoryLoader("/content/lang/",glob= '*.pdf',loader_cls=PyPDFLoader)
  documents=loader.load()
  text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
  texts=text_splitter.split_documents(documents)
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma.from_documents(texts,embeddings,persist_directory = './chroma_db')
  vector_db.persist()
  print("ChromaDB created and data saved")

  return vector_db
def setup_qa_chain(vector_db,llm):
    retriever=vector_db.retriever()
    prompt_templates= """ you are a companssionate mental health chatbot respond carefully:
    {context}
    user: {question}
    chatbot:"""
    prompt=PromptTemplate(template=prompt_templates,input_variables=['context','question'])

    qa_chain=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        chain_type_kwargs={'prompt':prompt}
    )
    return qa_chain


def main():
  print("initializing chatbot.....")
  llm=initialize_llm()

  db_path="/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db=create_vector_db()
  else:
     embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
     vector_db=Chroma(persist_directory=db_path,embedding_function=embeddings)
     qa_chain=setup_qa_chain(vector_db,llm)

  while True:
    query=input("\nHuman:")
    if query.lower() == "exit":
      print("Chatbot: Take Care of yourself,Goodbye!")
      break
    response= qa_chain.run(query)
    print(f"Chatbot:{response}")

if __name__=="__main__":
  main()

initializing chatbot.....


KeyboardInterrupt: Interrupted by user

In [ ]:
model

In [9]:
create_vector_db()

<ipython-input-8-f5b4740b1412>:20: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

ChromaDB created and data saved


<ipython-input-8-f5b4740b1412>:22: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [35]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import gradio as gr

def initialize_llm():
  llm=ChatGroq(
    temperature =0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
  return llm
def create_vector_db():
  loader =DirectoryLoader("/content/lang/",glob= '*.pdf',loader_cls=PyPDFLoader)
  documents=loader.load()
  text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
  texts=text_splitter.split_documents(documents)
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma.from_documents(texts,embeddings,persist_directory = './chroma_db')
  vector_db.persist()
  print("ChromaDB created and data saved")

  return vector_db
  def setup_qa_chain(vector_db,llm):
    retirver=vector_db.as_retriever()
    prompt_templates= """ you are a companssionate mental health chatbot respond carefully:
    {context}
    user: {question}
    chatbot:"""
    prompt=PromptTemplate(template=prompt_templates,input_variables=['context','question'])

    qa_chain=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retirver,
        chain_type_kwargs={'prompt':prompt}
    )
    return qa_chain


def main():
  print("initializing chatbot.....")
  llm=initialize_llm()

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
  llm=ChatGroq(
    temperature =0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
  return llm
def create_vector_db():
  loader =DirectoryLoader("/content/lang/",glob= '*.pdf',loader_cls=PyPDFLoader)
  documents=loader.load()
  text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
  texts=text_splitter.split_documents(documents)
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma.from_documents(texts,embeddings,persist_directory = './chroma_db')
  vector_db.persist()
  print("ChromaDB created and data saved")

  return vector_db
def setup_qa_chain(vector_db,llm):
    retriever=vector_db.as_retriever()
    prompt_templates= """ you are a companssionate mental health chatbot respond carefully:
    {context}
    user: {question}
    chatbot:"""
    prompt=PromptTemplate(template=prompt_templates,input_variables=['context','question'])

    qa_chain=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        chain_type_kwargs={'prompt':prompt}
    )
    return qa_chain


def main():
  print("initializing chatbot.....")
  llm=initialize_llm()

  db_path="/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db=create_vector_db()
  else:
     embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
     vector_db=Chroma(persist_directory=db_path,embedding_function=embeddings)
  qa_chain=setup_qa_chain(vector_db,llm)

  while True:
    query=input("\nHuman:")
    if query.lower() == "exit":
      print("Chatbot: Take Care of yourself,Goodbye!")
      break
    response= qa_chain.run(query)
    print(f"Chatbot:{response}")

if __name__=="__main__":
  main()
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma(persist_directory=db_path,embedding_function=embeddings)
  qa_chain=setup_qa_chain(vector_db,llm)

  while True:
    query=input("\nHuman:")
    if query.lower() == "exit":
      print("Chatbot: Take Care of yourself,Goodbye!")
      break
    response= qa_chain.run(query)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
  llm=ChatGroq(
    temperature =0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
  return llm
def create_vector_db():
  loader =DirectoryLoader("/content/lang/",glob= '*.pdf',loader_cls=PyPDFLoader)
  documents=loader.load()
  text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
  texts=text_splitter.split_documents(documents)
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma.from_documents(texts,embeddings,persist_directory = './chroma_db')
  vector_db.persist()
  print("ChromaDB created and data saved")

  return vector_db
def setup_qa_chain(vector_db,llm):
    retriever=vector_db.retriever()
    prompt_templates= """ you are a companssionate mental health chatbot respond carefully:
    {context}
    user: {question}
    chatbot:"""
    prompt=PromptTemplate(template=prompt_templates,input_variables=['context','question'])

    qa_chain=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        chain_type_kwargs={'prompt':prompt}
    )
    return qa_chain



print("initializing chatbot.....")
llm=initialize_llm()

db_path="/content/chroma_db"

if not os.path.exists(db_path):
    vector_db=create_vector_db()
else:
  embeddings=HuggingFaceBgeEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma(persist_directory=db_path,embedding_function=embeddings)
qa_chain=setup_qa_chain(vector_db,llm)

def chatbot_response(user_input,history = []):
  if not user_input.strip():
    return "Please Provide a valid input",history
  response =qa_chain(user_input)
  history.append((user_input,responce))
  return"",history

with gr,Blocks(theme= "Respair/Shiki@1.2.1") as app:
  chatbot=gr.ChatInterface(fn =chatbot_response,title ="Mental health chatBot")

app.lauch()


initializing chatbot.....


KeyboardInterrupt: Interrupted by user